3'UTR Python calculations


In [ ]:
def genes_extractor(path):
    all_genes = {}
    with open(path, mode = "r") as data:
        line  = data.readline().strip()
        line = data.readline().strip()
        while line !="":
            line = line.split("\t")
            genes = line[12].split("/")
            for i in genes:
                if not i in all_genes:
                    all_genes[i] = ""
            line = data.readline().strip()
    result = list(all_genes.keys())
    return result

In [ ]:
genes = genes_extractor("../data/ComRES/GO_ORA_cytoplasmic_translation.tsv")
with open("../data/transl_genes.txt", mode = "w") as out:
    print(*genes, sep = "\n", end = "", file = out)

In [ ]:
import scipy.stats
from statsmodels.stats.multitest import fdrcorrection


def codon_medians (pandas_input):
    uga = pandas_input[pandas_input["Codon"] == "TGA"]["Length"].tolist()
    uaa = pandas_input[pandas_input["Codon"] == "TAA"]["Length"].tolist()
    uag = pandas_input[pandas_input["Codon"] == "TAG"]["Length"].tolist()   
    
    uaa_uga = scipy.stats.median_test(uaa, uga).pvalue
    uaa_uag = scipy.stats.median_test(uaa, uag).pvalue
    uag_uga = scipy.stats.median_test(uag, uga).pvalue
    all_tog = scipy.stats.median_test(uag, uga,uaa).pvalue

    print(f"UAA-UAG = {uaa_uag}")
    print(f"UAA-UGA = {uaa_uga}")
    print(f"UAG-UGA = {uag_uga}")
    print(fdrcorrection([uaa_uag, uaa_uga, uag_uga]))

    return None 

In [ ]:
hum_gc = pd.read_csv("../data/GENCODE.tsv", sep = "\t", names = ["Transcript_ID", "Codon", "Length"])
hum_mane = pd.read_csv("../data/MANE.tsv", sep = "\t")
codon_medians(hum_gc)
codon_medians(hum_mane)